# Hidden Tests

In this file, the hidden tests for all the rubric points are to be described. The tests for the individual rubric points are enclosed within `# BEGIN <rubric_point>` and `# END <rubric_point>` NBConvert cells. `hidden_tests.py` works by executing the contents of those cells between those two tags for each `<rubric_point>`. In order to initialize variables, `hidden_tests.py` also executes all code within `BEGIN` and `END` tags that appear before the `original` test.

Code that is not enclosed within `BEGIN` and `END` tags are not executed by `hidden_tests.py`. They are used for generating the hidden datasets.

In [1]:
from hidden_tests import *
import otter_tests.gen_public_tests as gen_public_tests
import os, csv, json, copy, shutil
import random
import numpy as np

In [2]:
DIRECTORY = '..'
FILE = 'p4.ipynb'

In [3]:
results = {}

In [4]:
deductions = {}
rubric = parse_rubric_file(os.path.join(DIRECTORY, "rubric.md"))
directories = get_directories(rubric)
comments = get_all_comments(directories)

In [5]:
def write_readme(data, write_path):
    """write_readme(data, write_path) writes the contents of `data` into the README.txt file `write_path`"""
    f = open(write_path, encoding='utf-8')
    rubric_point = f.read().split("\n")[0].strip(" \n")
    f.close()
    
    f = open(write_path, 'w', encoding='utf-8')
    f.write(rubric_point + "\n\n" + data)
    f.close()

## Variables

Useful variables that are used by many rubric tests can be stored here. The contents of this tag will be executed before each rubric test, so these variables get initialized before each rubric test.

`verify_fn_defn` defines the function `verify_fn` which is used for verifying if the function `expected` and `actual` have the same outputs for all permutations of inputs from `var_lists`.

In [6]:
verify_fn_defn = """
def verify_fn(expected, actual, var_inputs, test_format):
    for var in var_inputs:
        try:
            actual_val = actual(*var)
        except Exception as e:
            output = "%s results: " % actual.__name__
            output += "%s error enountered on %s%s" % (type(e).__name__, actual.__name__, repr(var))
            return output
        expected_val = expected(*var)
        check = public_tests.compare(expected_val, actual_val, test_format)
        if check != public_tests.PASS:
            output = "%s results: " % actual.__name__
            output += "%s%s output: %s" % (actual.__name__, repr(var), check)
            return output
    return "%s results: All test cases passed!" % actual.__name__"""

`function_dependencies_functions` stores the previously defined functions that each function definition invokes. This variable is used for rubric points that the logical correctness of functions as well as those that check whether a required function is used. For these rubric points, when we test a particular function, we use `function_dependencies_functions` to ensure that all the functions that it depends on are replaced with logically correct versions. This helps isolate the issue with the functions.

In [7]:
function_dependencies_functions = {}
function_dependencies_functions['damage'] = []
function_dependencies_functions['type_bonus'] = []
function_dependencies_functions['get_num_types'] = []
function_dependencies_functions['effective_damage'] = ['get_num_types', 'type_bonus', 'damage']
function_dependencies_functions['num_hits'] = ['damage', 'get_num_types', 'type_bonus', 'effective_damage']
function_dependencies_functions['battle'] = ['num_hits', 'get_num_types', 'damage', 'type_bonus', 'effective_damage']
function_dependencies_functions['friendship_score'] = []

`function_dependencies_data_structures` stores the previously defined data structures that each function definition invokes. This variable is used for rubric points that the logical correctness of functions as well as those that check whether a required function is used. For these rubric points, when we test a particular function, we use `function_dependencies_data_structures` to ensure that all the data structures that it depends on are replaced with logically correct versions. This helps isolate the issue with the functions.

In [8]:
function_dependencies_data_structures = {}
function_dependencies_data_structures['damage'] = []
function_dependencies_data_structures['type_bonus'] = []
function_dependencies_data_structures['get_num_types'] = []
function_dependencies_data_structures['effective_damage'] = []
function_dependencies_data_structures['num_hits'] = []
function_dependencies_data_structures['battle'] = []
function_dependencies_data_structures['friendship_score'] = []

`data_structure_dependencies_functions` stores the previously defined functions that each data structure definition invokes. This variable is used for rubric points that the logical correctness of functions as well as those that check whether a required data structure is used. For these rubric points, when we test a particular data structure, we use `data_structure_dependencies_functions` to ensure that all the functions that it depends on are replaced with logically correct versions. This helps isolate the issue with the data structures.

In [9]:
data_structure_dependencies_functions = {}

`data_structure_dependencies_data_structures` stores the previously defined data structures that each data structure definition accesses. This variable is used for rubric points that the logical correctness of data structures as well as those that check whether a required data structure is used. For these rubric points, when we test a particular data structure, we use `data_structure_dependencies_data_structures` to ensure that all the data structures that it depends on are replaced with logically correct versions. This helps isolate the issue with the data structures.

In [10]:
data_structure_dependencies_data_structures = {} 

## Functions

Useful functions that are used by many rubric tests can be stored here. The contents of this tag will be executed before each rubric test, so these function definitions get initialized before each rubric test.

`replace_with_false_function` replaces the given `function` with the **false version** of the function, and also replaces all **dependent** functions and data structures with their **true versions**.

In [11]:
def replace_with_false_function(nb, function, false_function):
    nb = replace_defn(nb, function, false_function)
    
    for dependent in function_dependencies_functions.get(function, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in function_dependencies_data_structures.get(function, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
    return nb

`replace_with_false_data_structure` replaces the given `data_structure` with the **false** version of the data structure, and also replaces all **dependent** functions and data structures with their **true versions**.

In [12]:
def replace_with_false_data_structure(nb, data_structure, false_data_structure):
    idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (data_structure))[-1]
    if idx == None:
        idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
    nb = inject_code(nb, idx, false_data_structure)
    nb = remove_initializations(nb, data_structure, start=idx+1)
    
    for dependent in data_structure_dependencies_functions.get(data_structure, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in data_structure_dependencies_data_structures.get(data_structure, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
    return nb

`get_test_text` returns test code that can be readily injected into the notebook. The input should be some code that updates the variable `test_output` and sets its value to be `"All test cases passed!"` when the conditions for passing the rubric test are met. This function will place this code inside a wrapper than ensures that it does not crash the student notebook during execution and also makes the output parsable.

In [13]:
def get_test_text(qnum, test_code):
    test_text = "\"\"\"grader.check('%s')\"\"\"\n\n" % (qnum)
    test_text += "test_output = '%s results: Test crashed!'\n" % (qnum)
    test_text += add_try_except(test_code)
    test_text += "\nprint(test_output)"
    return test_text

`inject_function_logic_check` injects code into the `nb` that detects whether `function` outputs the same as the **true version** of that function (all dependent functions and data structures are also replaced with their **true versions**) on all combinations of inputs from `var_lists`. The comparison between the outputs is performed assuming that the format of the answers is `test_format`.

In [14]:
def inject_function_logic_check(nb, function, var_inputs_code, test_format="TEXT_FORMAT"):
    for dependent in function_dependencies_functions.get(function, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in function_dependencies_data_structures.get(function, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
        
    code = replace_call(true_functions[function], function, "true_"+function)
    code += "\n\n" + verify_fn_defn
    nb = inject_code(nb, len(nb['cells']), code)
    test_code = var_inputs_code + "\n"
    test_code += "test_output = verify_fn(true_%s, %s, var_inputs, '%s')" % (function, function, test_format)
    code = get_test_text(function, test_code)
    nb = inject_code(nb, len(nb['cells']), code)
    return nb

`inject_data_structure_check` injects code into the `nb` that detects whether `data_structure` has the same value as the **true version** of that data structure (all dependent functions and data structures are also replaced with their **true versions**). The comparison between the outputs is performed assuming that the format of the answers is `test_format`.

In [15]:
def inject_data_structure_check(nb, data_structure, test_format="TEXT_FORMAT"):
    for dependent in data_structure_dependencies_functions.get(data_structure, []):
        nb = replace_defn(nb, dependent, true_functions[dependent])
    for dependent in data_structure_dependencies_data_structures.get(data_structure, []):
        idx = find_all_cell_indices(nb, "code", "grader.check('%s')" % (dependent))[-1]
        if idx == None:
            idx = find_all_cell_indices(nb, "markdown", "**Question 1:**")[-1]
        nb = inject_code(nb, idx, true_data_structures[dependent])
        nb = remove_initializations(nb, dependent, start=idx+1)
        
    code = "import copy\n%s = copy.deepcopy(%s)\n\n" % (data_structure, data_structure)
    code += replace_variable(true_data_structures[data_structure], data_structure, "true_"+data_structure)
    nb = inject_code(nb, len(nb['cells']), code)
    
    test_code = "test_output = '%s results: '" % (data_structure)
    test_code += "+ public_tests.compare(true_%s, %s, '%s')" % (data_structure, data_structure, test_format)
    code = get_test_text(data_structure, test_code)
    nb = inject_code(nb, len(nb['cells']), code)
    return nb

## Random Data Generation

Here, functions are defined that can generate **random** data that is in the correct format.

**Warning:** This is the most complex function in the file, and is likely to have some bugs in it. So, **verify** this function **carefully**. The following **requirements** for this function **will not** be met by the function generated by GPT, it is **your responsibility** to modify the function so as to meet these requirements. Otherwise, the datasets are unlikely to produce interesting outputs for the project questions.

In [16]:
import os
import csv
import random

def random_data(directory, n=1000):
    """
    Generate random datasets for pokemon_stats.csv and type_effectiveness_stats.csv as described.
    The pokemon names are taken from the hidden original pokemon_stats.csv file.
    Other values are randomized as per the rules given.
    """
    
    # Delete existing files if they exist
    pokemon_stats_path = os.path.join(directory, 'pokemon_stats.csv')
    type_effectiveness_stats_path = os.path.join(directory, 'type_effectiveness_stats.csv')
    
    if os.path.exists(pokemon_stats_path):
        os.remove(pokemon_stats_path)
    
    if os.path.exists(type_effectiveness_stats_path):
        os.remove(type_effectiveness_stats_path)

    # Get Pokemon names from the hidden original file
    original_pokemon_stats_path = os.path.join(DIRECTORY, 'hidden', 'original', 'pokemon_stats.csv')
    original_type_effectiveness_stats_path = os.path.join(DIRECTORY, 'hidden', 'original', 'type_effectiveness_stats.csv')
    with open(original_pokemon_stats_path, 'r', encoding='utf-8') as original_file:
        reader = csv.reader(original_file)
        next(reader)  # Skip header
        pokemon_names = [row[1] for row in reader]  # Gather all Pokemon names
    
    # Get regions from the hidden original file
    regions = set()
    with open(original_pokemon_stats_path, 'r', encoding='utf-8') as original_file:
        reader = csv.DictReader(original_file)
        for row in reader:
            regions.add(row['Region'])
    regions = list(regions)

    # Get types from the type_effectiveness_stats.csv file
    types = []
    with open(original_type_effectiveness_stats_path, 'r', encoding='utf-8') as types_file:
        reader = csv.reader(types_file)
        types = next(reader)[1:]  # Skip index column

    # Generate pokemon_stats.csv
    with open(pokemon_stats_path, 'w', encoding='utf-8', newline='') as ps_file:
        writer = csv.writer(ps_file)
        headers = ['Name', 'Attack', 'Defense', 'HP', 'Region', 'Sp. Atk', 'Sp. Def', 'Speed', 'Type 1', 'Type 2']
        writer.writerow([''] + headers)
        for i, name in enumerate(pokemon_names):
            type1 =  random.choice(types)
            type2 = 'DNE'
            type2_choice = random.randint(1, 4)
            if type2_choice != 1:
                type2 = random.choice(types)
                while type2 == type1:
                    type2 = random.choice(types)
            row = [
                name,  # Name
                random.randint(1, 256),  # Attack
                random.randint(1, 256),  # Defense
                random.randint(1, 256),  # HP
                random.choice(regions),  # Region
                random.randint(1, 256),  # Sp. Atk
                random.randint(1, 256),  # Sp. Def
                random.randint(1, 256),  # Speed
                type1, # Type 1
                type2 # Type 2
            ]
            writer.writerow([i] + row)

    # Generate type_effectiveness_stats.csv
    with open(type_effectiveness_stats_path, 'w', encoding='utf-8', newline='') as tes_file:
        writer = csv.writer(tes_file)
        writer.writerow([''] + types)
        for type1 in types:
            row = [type1] + [random.choice([0.0] + [0.5]*4 + [1.0]*8 + [2.0]*4) for _ in types]
            writer.writerow(row)

## True Functions

Here, the **correct** versions of all functions that are defined in the notebook are stored. These functions are compared against the functions in the student notebook to check for their correctness.

In [17]:
true_functions = {}

In [18]:
true_functions['damage'] = """
import project

def damage(attacker, defender):
    physical_damage = 10 * project.get_attack(attacker) / project.get_defense(defender)
    special_damage = 10 * project.get_sp_atk(attacker) / project.get_sp_def(defender)
    if physical_damage > special_damage:
        return physical_damage
    else:
        return special_damage"""

In [19]:
true_functions['type_bonus'] = """
import project

def type_bonus(attack_type, defender):
    defender_type1 = project.get_type1(defender)
    defender_type2 = project.get_type2(defender)
    bonus = project.get_type_effectiveness(attack_type, defender_type1)
    if defender_type2 == \'DNE\':
        return bonus
    else:
        bonus = bonus * project.get_type_effectiveness(attack_type, defender_type2)
        return bonus"""

In [20]:
true_functions['get_num_types'] = """
import project

def get_num_types(pkmn):
    if project.get_type2(pkmn) == \'DNE\':
        return 1
    return 2"""

In [21]:
true_functions['effective_damage'] = """
import project

def effective_damage(attacker, defender):
    if get_num_types(attacker) == 1:
        return damage(attacker, defender) * type_bonus(project.get_type1(attacker), defender)
    else:
        bonus1 = type_bonus(project.get_type1(attacker), defender)
        bonus2 = type_bonus(project.get_type2(attacker), defender)
        return damage(attacker, defender) * max(bonus1, bonus2)"""

In [22]:
true_functions['num_hits'] = """
import project
import math

def num_hits(attacker, defender):
    if effective_damage(attacker, defender) == 0:
        return \'infinitely many\'
    return math.ceil(project.get_hp(defender) / effective_damage(attacker, defender))"""

In [23]:
true_functions['battle'] = """
import project

def battle(pkmn1, pkmn2):
    if num_hits(pkmn1, pkmn2) == \'infinitely many\':
        if num_hits(pkmn2, pkmn1) == \'infinitely many\':
            return \'Draw\'
        else:
            return pkmn2
    elif num_hits(pkmn2, pkmn1) == \'infinitely many\':
        return pkmn1
    elif num_hits(pkmn1, pkmn2) > num_hits(pkmn2, pkmn1):
        return pkmn2
    elif num_hits(pkmn1, pkmn2) < num_hits(pkmn2, pkmn1):
        return pkmn1
    elif project.get_speed(pkmn1) > project.get_speed(pkmn2):
        return pkmn1
    elif project.get_speed(pkmn1) < project.get_speed(pkmn2):
        return pkmn2
    else:
        return \'Draw\'"""

In [24]:
true_functions['friendship_score'] = """
import project

def hidn_get_stat_total(pkmn):
    stat_total = project.get_attack(pkmn) + project.get_defense(pkmn)
    stat_total += project.get_sp_atk(pkmn) + project.get_sp_def(pkmn)
    stat_total += project.get_hp(pkmn) + project.get_speed(pkmn)
    return stat_total

def friendship_score(pkmn1, pkmn2):
    friendship = 0
    pkmn1_region = project.get_region(pkmn1)
    pkmn2_region = project.get_region(pkmn2)
    if pkmn1_region == pkmn2_region:
        friendship += 1
    if abs(hidn_get_stat_total(pkmn1) - hidn_get_stat_total(pkmn2)) <= 20:
        friendship += 1
    pkmn1_type1 = project.get_type1(pkmn1)
    pkmn1_type2 = project.get_type2(pkmn1)
    pkmn2_type1 = project.get_type1(pkmn2)
    pkmn2_type2 = project.get_type2(pkmn2)
    if pkmn1_type1 == pkmn2_type1:
        if pkmn1_type2 != \'DNE\' and pkmn1_type2 == pkmn2_type2:
            friendship += 3
        else:
            friendship += 1
    elif pkmn1_type2 != \'DNE\' and pkmn1_type2 == pkmn2_type2:
        friendship += 1
    return friendship"""

## True Data Structures

Here, the **correct** versions of all data structures that are defined in the notebook are stored. These data structures are compared against the data structures in the student notebook to check for their correctness.

In [25]:
true_data_structures = {}

## Original

The original test simply runs the student's notebook as it is (after removing cells with syntax errors, and performing other clean-up). This helps us detect if the student failed any public tests.

In [26]:
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))

results['original'] = parse_nb(run_nb(nb, os.path.join(DIRECTORY, "hidden", "original", FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


## Hardcode

The hardcode tests run the student's notebook on different datasets. However, `public_tests.py` remains unchanged. So, if the answers are hardcoded in the student's notebook, we expect their code to still pass the public tests on all the different datasets. If their code fails any one of the different hardcode datasets, we take that to mean that the answer is not hardcoded.

In [27]:
for i in range(1, 6):
    if str(i) in os.listdir(os.path.join(DIRECTORY, "hidden", "hardcode")):
        continue
    shutil.copytree(os.path.join(DIRECTORY, "hidden", "original"), os.path.join(DIRECTORY, "hidden", "hardcode", str(i)))
    
for i in range(1, 6):
    f = open(os.path.join(DIRECTORY, "hidden", "hardcode", str(i), 'README.txt'), 'w', encoding='utf-8')
    f.write("hardcode: %d" % (i))
    f.close()

In [31]:
for subdirectory in os.listdir(os.path.join(DIRECTORY, "hidden", "hardcode")):
    path = os.path.join(DIRECTORY, "hidden", "hardcode", subdirectory)
    if not os.path.isdir(path):
        continue
    good_dataset = False
    while not good_dataset:
        if os.path.exists(os.path.join(path, FILE)):
            nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
        hardcode_results = parse_nb(run_nb(nb, os.path.join(path, FILE)))
        good_dataset = True
        for qnum in hardcode_results:
            if qnum.startswith('q') and hardcode_results[qnum] == 'All test cases passed!':
                print(qnum + ' failed!')
                good_dataset = False
                break
        if not good_dataset:
            random_data(path)
    print(subdirectory + ' done!')

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


1 done!


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


4 done!


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


3 done!


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


2 done!


0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


5 done!


In [33]:
for hardcode in os.listdir(os.path.join(DIRECTORY, "hidden", "hardcode")):
    if hardcode == ".DS_Store":
        continue
    nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
    results['hardcode: ' + hardcode] = parse_nb(run_nb(nb, os.path.join(DIRECTORY, "hidden", "hardcode", hardcode, FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0

## Rubric Tests

The tests for the rubric points will be defined below. Only the code inside the tags will be executed by `hidden_tests.py`, so the code outside the tags are used for generating the hidden datasets in the first place.

### Instructions for creating rubric tests:

Functions inside `hidden_tests.py` can be used to modify the student notebook, before executing and parsing the outputs. It is recommended that before trying to create rubric tests, a user goes through all the functions inside `hidden_tests.py` first. Here is a list of commonly used functions that will be most useful:

* **`read_nb`**: `read_nb(file)` **reads** a `file` in the `.ipynb` file format and returns a `nb`.
* **`run_nb`**: `run_nb(nb, file)` **executes** `nb` at the location `file` and **writes** the contents back into `file`.
* **`parse_nb`**: `parse_nb(nb)` read the contents of a student `nb` and **extracts** all graded questions and answers.
* **`truncate_nb`**: `truncate_nb(nb, start, end)` takes in a `nb`, and returns a **sliced** notebook between the cells indexed `start` and `end`.
* **`find_all_cell_indices`**: `find_all_cell_indices(nb, cell_type, marker)` returns **all** the indices in `nb` of cell type `cell_type` that **contains** the `marker` in its source.
* **`inject_code`**: `inject_code(nb, idx, code)` creates a **new** code cell in `nb` **after** the index `idx` with `code` in it.
* **`count_defns`**: `count_defns(nb, func_name)` **counts** the number of times `func_name` is defined in the `nb`.
* **`replace_defn`**: `replace_defn(nb, func_name, new_defn)` **replaces** the definition of `func_name` in `nb` with `new_defn`.
* **`replace_call`**: `replace_call(text, func_name, new_name)` **replaces** all **calls** and definition **names** to `func_name` with `new_name` in `text`.
* **`find_code`**: `find_code(nb, target)` returns the **number** of times that the **text** `target` appears in a code cell in `nb`.
* **`replace_code`**: `replace_code(nb, target, new_code, start, end)` **replaces** all instances of the **text** `target` in a code cell between the indices `start` and `end` with the **text** `new_code`.
* **`add_try_except`**: `add_try_except(text)` adds a (bare) **try/except block** around any given block of code.
* **`detect_restart_and_run_all`**: `detect_restart_and_run_all(nb)` flags if any **non-empty code cell** in `nb` is **not executed**.
* **`detect_imports`**: `detect_imports(nb)` returns a list of **all** the **import** statements in the `nb`.
* **`detect_ast_objects`**: `detect_ast_objects(nb, objects)` returns a dict of **all** cells in the `nb` with the **ast objects** `objects` in them.
* **`get_first_plot`**: `get_first_plot(nb, image_file)` returns the first **image** found in the output of a code cell in `nb`, and also stores it in `image_file` for reference.
* **`get_label_plot`**: `get_label_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing just the **label** at the location indicated by `kind` - `"left"`, `"right"`, `"top"`, or `"bottom"`.
* **`get_without_label_plot`**: `get_without_label_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing everything **except** the **label** at the location indicated by `kind` - `"left"`, `"right"`, `"top"`, or `"bottom"`.
* **`get_ticks_plot`**: `get_ticks_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing just the **ticks** at the location indicated by `kind` - `"left"`, or `"bottom"`.
* **`get_without_ticks_plot`**: `get_without_ticks_plot(plot, kind)` **crops** the `plot` and returns returns a plot containing everything **except** the **ticks** at the location indicated by `kind` - `"left"`, or `"bottom"`.
* **`get_bounding_box_plot`**: `get_bounding_box_plot(plot)` **crops** the `plot` and returns returns a plot containing just the **bounding box** of the plot.
* **`check_text_in_plot`**: `check_text_in_plot(plot, expected_text)` checks if the `expected_text` is in the `plot`, and returns both the **missing** and the **extra** text in the given `plot`.

### damage: function output is incorrect when the `attacker` needs to choose its physical attack

In [34]:
rubric_item = 'damage: function output is incorrect when the `attacker` needs to choose its physical attack'
readme_text = """Confirm that your function chooses the correct
attack mode by comparing physical and special
damage. Ensure you have correctly used the
provided functions to compute damage values and
are comparing them accurately to return the
greater of the two. Review the calculation and
conditional statement logic."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [35]:
rubric_item = 'damage: function output is incorrect when the `attacker` needs to choose its physical attack'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('damage')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attacker in project.__pokemon__:
    for _defender in project.__pokemon__:
        _physical_damage = 10 * project.get_attack(_attacker) / project.get_defense(_defender)
        _special_damage = 10 * project.get_sp_atk(_attacker) / project.get_sp_def(_defender)
        if _physical_damage >= _special_damage:
            var_inputs.append((_attacker, _defender))
'''
nb = inject_function_logic_check(nb, 'damage', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### damage: function output is incorrect when the `attacker` needs to choose its special attack

In [36]:
rubric_item = 'damage: function output is incorrect when the `attacker` needs to choose its special attack'
readme_text = """Confirm that your function chooses the correct
attack mode by comparing physical and special
damage. Ensure you have correctly used the
provided functions to compute damage values and
are comparing them accurately to return the
greater of the two. Review the calculation and
conditional statement logic."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [37]:
rubric_item = 'damage: function output is incorrect when the `attacker` needs to choose its special attack'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('damage')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attacker in project.__pokemon__:
    for _defender in project.__pokemon__:
        _physical_damage = 10 * project.get_attack(_attacker) / project.get_defense(_defender)
        _special_damage = 10 * project.get_sp_atk(_attacker) / project.get_sp_def(_defender)
        if _physical_damage <= _special_damage:
            var_inputs.append((_attacker, _defender))
'''
nb = inject_function_logic_check(nb, 'damage', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q1: correct arguments are not passed to `damage` function

In [38]:
rubric_item = 'q1: correct arguments are not passed to `damage` function'
readme_text = """The test checks if you passed the correct
arguments to the `damage` function. To fix your
code, ensure that the `damage` function must be
called with the exact Pokémon names: 'Tinkaton'
and 'Arcanine', in that order."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [39]:
rubric_item = 'q1: correct arguments are not passed to `damage` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q1')")[-1])

new_damage = '''
def damage(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'damage', new_damage)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [40]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q2: correct arguments are not passed to `damage` function

In [41]:
rubric_item = 'q2: correct arguments are not passed to `damage` function'
readme_text = """The test checks if you passed the correct
arguments to the `damage` function. To fix your
code, ensure that the `damage` function must be
called with the exact Pokémon names: 'Lucario'
and 'Klawf', in that order."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [42]:
rubric_item = 'q2: correct arguments are not passed to `damage` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q2')")[-1])

new_damage = '''
def damage(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'damage', new_damage)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [43]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### type_bonus: function output is incorrect when the `defender` has only one type

In [44]:
rubric_item = 'type_bonus: function output is incorrect when the `defender` has only one type'
readme_text = """Check if your function correctly handles cases
where the `defender` has only one type. Ensure
that it correctly uses the `project.get_type1()`
and does not apply a second type multiplier if
`defender_type2` is `'DNE'`. Review for logic
errors that could misinterpret a single-type
defender."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [45]:
rubric_item = 'type_bonus: function output is incorrect when the `defender` has only one type'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('type_bonus')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attack_type in project.__effectiveness__:
    for _defender in project.__pokemon__:
        if project.get_type2(_defender) == "DNE":
            var_inputs.append((_attack_type, _defender))
'''

nb = inject_function_logic_check(nb, 'type_bonus', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### type_bonus: function output is incorrect when the `defender` has two types

In [46]:
rubric_item = 'type_bonus: function output is incorrect when the `defender` has two types'
readme_text = """Check if your function correctly handles cases
where the `defender` has two types. Ensure
that it correctly uses the `project.get_type1()`
and applies a second type multiplier if
`defender_type2` is not `'DNE'`. Review for logic
errors that could misinterpret a dual-type
defender."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [47]:
rubric_item = 'type_bonus: function output is incorrect when the `defender` has two types'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('type_bonus')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attack_type in project.__effectiveness__:
    for _defender in project.__pokemon__:
        if project.get_type2(_defender) != "DNE":
            var_inputs.append((_attack_type, _defender))
'''

nb = inject_function_logic_check(nb, 'type_bonus', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q3: correct arguments are not passed to `type_bonus` function

In [48]:
rubric_item = 'q3: correct arguments are not passed to `type_bonus` function'
readme_text = """Ensure the `type_bonus` function is called with
the correct string arguments for the Pokémon type
and name. Review the function parameters and the
expected arguments."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [49]:
rubric_item = 'q3: correct arguments are not passed to `type_bonus` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q3')")[-1])

new_type_bonus = '''
def type_bonus(attack_type, defender):
    return [attack_type, defender]
'''
nb = replace_with_false_function(nb, 'type_bonus', new_type_bonus)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [50]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q4: correct arguments are not passed to `type_bonus` function

In [51]:
rubric_item = 'q4: correct arguments are not passed to `type_bonus` function'
readme_text = """Ensure the `type_bonus` function is called with
the correct string arguments for the Pokémon type
and name. Review the function parameters and the
expected arguments."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [52]:
rubric_item = 'q4: correct arguments are not passed to `type_bonus` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q4')")[-1])

new_type_bonus = '''
def type_bonus(attack_type, defender):
    return [attack_type, defender]
'''
nb = replace_with_false_function(nb, 'type_bonus', new_type_bonus)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [53]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### get_num_types: function logic is incorrect

In [54]:
rubric_item = 'get_num_types: function logic is incorrect'
readme_text = """Check the implementation of `get_num_types` to
ensure it properly accounts for all Pokemon types,
including 'DNE'. Consider the function's logic and
make sure it returns the correct number of types
for each input it is given."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [55]:
rubric_item = 'get_num_types: function logic is incorrect'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('get_num_types')")[-1])

var_inputs_code = '''
import project
var_inputs = [(_pkmn,) for _pkmn in list(project.__pokemon__.keys())]
'''
nb = inject_function_logic_check(nb, 'get_num_types', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### effective_damage: `get_num_types` function is not used by `effective_damage`

In [56]:
rubric_item = 'effective_damage: `get_num_types` function is not used by `effective_damage`'
readme_text = """Ensure `get_num_types` is called by
`effective_damage` to determine the attacker's
number of types. Check if you inadvertently
re-implemented its logic instead of calling the
function. Consider revisiting function calls and
modularity concepts."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [57]:
rubric_item = 'effective_damage: `get_num_types` function is not used by `effective_damage`'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('effective_damage')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attacker in project.__pokemon__:
    for _defender in project.__pokemon__:
        var_inputs.append((_attacker, _defender))
'''
nb = inject_function_logic_check(nb, 'effective_damage', var_inputs_code, "TEXT_FORMAT")

false_get_num_types = '''
def get_num_types(pkmn):
    return 1
'''
nb = replace_with_false_function(nb, 'get_num_types', false_get_num_types)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### effective_damage: function output is incorrect when the `attacker` has only one type

In [58]:
rubric_item = 'effective_damage: function output is incorrect when the `attacker` has only one type'
readme_text = """Ensure that your implementation of
`effective_damage` correctly handles cases where
the attacker has one type. Double-check your use
of `get_num_types` within the function. Consider
edge cases involving specific inputs not covered
by your local tests."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [59]:
rubric_item = 'effective_damage: function output is incorrect when the `attacker` has only one type'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('effective_damage')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attacker in project.__pokemon__:
    for _defender in project.__pokemon__:
        if project.get_type2(_attacker) == "DNE":
            var_inputs.append((_attacker, _defender))
'''
nb = inject_function_logic_check(nb, 'effective_damage', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### effective_damage: function output is incorrect when the `attacker` has two types

In [60]:
rubric_item = 'effective_damage: function output is incorrect when the `attacker` has two types'
readme_text = """Ensure that your implementation of
`effective_damage` correctly handles cases where
the attacker has two types. Double-check your use
of `get_num_types` within the function. Consider
edge cases involving specific inputs not covered
by your local tests."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [61]:
rubric_item = 'effective_damage: function output is incorrect when the `attacker` has two types'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('effective_damage')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attacker in project.__pokemon__:
    for _defender in project.__pokemon__:
        if project.get_type2(_attacker) != "DNE":
            var_inputs.append((_attacker, _defender))
'''
nb = inject_function_logic_check(nb, 'effective_damage', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q5: correct arguments are not passed to `effective_damage` function

In [62]:
rubric_item = 'q5: correct arguments are not passed to `effective_damage` function'
readme_text = """Check the arguments passed to `effective_damage`.
The function should be called with specific
parameters. Review the function's intended inputs."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [63]:
rubric_item = 'q5: correct arguments are not passed to `effective_damage` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q5')")[-1])

new_effective_damage = '''
def effective_damage(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'effective_damage', new_effective_damage)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [64]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q6: correct arguments are not passed to `effective_damage` function

In [65]:
rubric_item = 'q6: correct arguments are not passed to `effective_damage` function'
readme_text = """Check the arguments passed to `effective_damage`.
The function should be called with specific
parameters. Review the function's intended inputs."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [66]:
rubric_item = 'q6: correct arguments are not passed to `effective_damage` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q6')")[-1])

new_effective_damage = '''
def effective_damage(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'effective_damage', new_effective_damage)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [67]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q7: correct arguments are not passed to `effective_damage` function

In [68]:
rubric_item = 'q7: correct arguments are not passed to `effective_damage` function'
readme_text = """Check the arguments passed to `effective_damage`.
The function should be called with specific
parameters. Review the function's intended inputs."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [69]:
rubric_item = 'q7: correct arguments are not passed to `effective_damage` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q7')")[-1])

new_effective_damage = '''
def effective_damage(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'effective_damage', new_effective_damage)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [70]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### num_hits: function output is incorrect when the `attacker` can do non-zero effective damage to the `defender`

In [71]:
rubric_item = 'num_hits: function output is incorrect when the `attacker` can do non-zero effective damage to the `defender`'
readme_text = """Ensure your `num_hits` function correctly divides
HP by effective damage and rounds up. Use
`math.ceil`. Check for any logic errors that might
affect the outcome with various inputs."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [72]:
rubric_item = 'num_hits: function output is incorrect when the `attacker` can do non-zero effective damage to the `defender`'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('num_hits')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attacker in project.__pokemon__:
    for _defender in project.__pokemon__:
        if effective_damage(_attacker, _defender) != 0:
            var_inputs.append((_attacker, _defender))
'''
nb = inject_function_logic_check(nb, 'num_hits', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### num_hits: function output is incorrect when the `attacker` cannot do any damage to the `defender`

In [73]:
rubric_item = 'num_hits: function output is incorrect when the `attacker` cannot do any damage to the `defender`'
readme_text = """Ensure `num_hits` correctly handles cases where the
`attacker` can do zero effective damage to `defender`.
In this case, `num_hits` should return 'infinitely many'. 
Review handling of division by zero and conditions 
for effective damage."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [74]:
rubric_item = 'num_hits: function output is incorrect when the `attacker` cannot do any damage to the `defender`'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('num_hits')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _attacker in project.__pokemon__:
    for _defender in project.__pokemon__:
        if effective_damage(_attacker, _defender) == 0:
            var_inputs.append((_attacker, _defender))
'''
nb = inject_function_logic_check(nb, 'num_hits', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q8: correct arguments are not passed to `num_hits` function

In [75]:
rubric_item = 'q8: correct arguments are not passed to `num_hits` function'
readme_text = """The test ensures the `num_hits` function is called
with the correct arguments. Verify that the
arguments to `num_hits` match the order specified
in the function definition and that you are using
the correct Pokemon names as parameters."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [76]:
rubric_item = 'q8: correct arguments are not passed to `num_hits` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q8')")[-1])

new_num_hits = '''
def num_hits(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'num_hits', new_num_hits)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [77]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q9: correct arguments are not passed to `num_hits` function

In [78]:
rubric_item = 'q9: correct arguments are not passed to `num_hits` function'
readme_text = """The test ensures the `num_hits` function is called
with the correct arguments. Verify that the
arguments to `num_hits` match the order specified
in the function definition and that you are using
the correct Pokemon names as parameters."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [79]:
rubric_item = 'q9: correct arguments are not passed to `num_hits` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q9')")[-1])

new_num_hits = '''
def num_hits(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'num_hits', new_num_hits)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [80]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q10: correct arguments are not passed to `num_hits` function

In [130]:
rubric_item = 'q10: correct arguments are not passed to `num_hits` function'
readme_text = """The test ensures the `num_hits` function is called
with the correct arguments. Verify that the
arguments to `num_hits` match the order specified
in the function definition and that you are using
the correct Pokemon names as parameters."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [82]:
rubric_item = 'q10: correct arguments are not passed to `num_hits` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q10')")[-1])

new_num_hits = '''
def num_hits(attacker, defender):
    return [attacker, defender]
'''
nb = replace_with_false_function(nb, 'num_hits', new_num_hits)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [83]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### battle: function output is incorrect when the two Pokemon can do damage to each other and do not take the same number of hits to defeat each other

In [84]:
rubric_item = 'battle: function output is incorrect when the two Pokemon can do damage to each other and do not take the same number of hits to defeat each other'
readme_text = """Ensure that your `battle` function correctly
compares the number of hits each Pokemon can take
using the `num_hits` function. Verify that you are
not calling the functions more times than needed,
which may change the logic. Check for correct
comparison logic for the number of hits and speed,
if applicable."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [85]:
rubric_item = 'battle: function output is incorrect when the two Pokemon can do damage to each other and do not take the same number of hits to defeat each other'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('battle')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _pkmn1 in project.__pokemon__:
    for _pkmn2 in project.__pokemon__:
        if num_hits(_pkmn1, _pkmn2) == 'infinitely many':
            continue
        elif num_hits(_pkmn2, _pkmn1) == 'infinitely many':
            continue
        elif num_hits(_pkmn1, _pkmn2) == num_hits(_pkmn2, _pkmn1):
            continue
        var_inputs.append((_pkmn1, _pkmn2))
'''
nb = inject_function_logic_check(nb, 'battle', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### battle: function output is incorrect when the two Pokemon can do damage to each other but take the same number of hits to defeat each other

In [86]:
rubric_item = 'battle: function output is incorrect when the two Pokemon can do damage to each other but take the same number of hits to defeat each other'
readme_text = """The test checks for correct output when two
Pokemon can deal damage but take equal hits to
defeat each other. Verify your implementation of
`battle` and consider edge cases where the Pokemon
have the same number of hits to knock each other
out, ensuring your code accounts for speed as a
tiebreaker."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [87]:
rubric_item = 'battle: function output is incorrect when the two Pokemon can do damage to each other but take the same number of hits to defeat each other'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('battle')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _pkmn1 in project.__pokemon__:
    for _pkmn2 in project.__pokemon__:
        if num_hits(_pkmn1, _pkmn2) == 'infinitely many':
            continue
        elif num_hits(_pkmn2, _pkmn1) == 'infinitely many':
            continue
        elif num_hits(_pkmn1, _pkmn2) == num_hits(_pkmn2, _pkmn1):
            var_inputs.append((_pkmn1, _pkmn2))
'''
nb = inject_function_logic_check(nb, 'battle', var_inputs_code, "TEXT_FORMAT")

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### battle: function output is incorrect when one or more of the Pokemon cannot damage the other

In [88]:
rubric_item = 'battle: function output is incorrect when one or more of the Pokemon cannot damage the other'
readme_text = """The test indicated that your `battle` function
might not handle scenarios where one Pokemon
cannot damage another correctly. Verify your
implementation to ensure that it gracefully
addresses cases with 'infinitely many' hits and
compares the hits required by each Pokemon
accurately, taking into account their speed when
necessary."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [89]:
rubric_item = 'battle: function output is incorrect when one or more of the Pokemon cannot damage the other'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, 'code', "grader.check('battle')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _pkmn1 in project.__pokemon__:
    for _pkmn2 in project.__pokemon__:
        if num_hits(_pkmn1, _pkmn2) == 'infinitely many':
            var_inputs.append((_pkmn1, _pkmn2))
        elif num_hits(_pkmn2, _pkmn1) == 'infinitely many':
            var_inputs.append((_pkmn1, _pkmn2))        
'''
nb = inject_function_logic_check(nb, 'battle', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q11: correct arguments are not passed to `battle` function

In [90]:
rubric_item = 'q11: correct arguments are not passed to `battle` function'
readme_text = """The test verifies if the `battle` function is
called with the correct arguments. Please ensure
that the function is defined correctly and
receives the expected arguments. Review function
definition and argument passing in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [91]:
rubric_item = 'q11: correct arguments are not passed to `battle` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q11')")[-1])

new_battle = '''
def battle(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'battle', new_battle)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [92]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q12: correct arguments are not passed to `battle` function

In [93]:
rubric_item = 'q12: correct arguments are not passed to `battle` function'
readme_text = """The test verifies if the `battle` function is
called with the correct arguments. Please ensure
that the function is defined correctly and
receives the expected arguments. Review function
definition and argument passing in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [94]:
rubric_item = 'q12: correct arguments are not passed to `battle` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q12')")[-1])

new_battle = '''
def battle(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'battle', new_battle)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [95]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q13: correct arguments are not passed to `battle` function

In [96]:
rubric_item = 'q13: correct arguments are not passed to `battle` function'
readme_text = """The test verifies if the `battle` function is
called with the correct arguments. Please ensure
that the function is defined correctly and
receives the expected arguments. Review function
definition and argument passing in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [97]:
rubric_item = 'q13: correct arguments are not passed to `battle` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q13')")[-1])

new_battle = '''
def battle(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'battle', new_battle)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [98]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q14: correct arguments are not passed to `battle` function

In [99]:
rubric_item = 'q14: correct arguments are not passed to `battle` function'
readme_text = """The test verifies if the `battle` function is
called with the correct arguments. Please ensure
that the function is defined correctly and
receives the expected arguments. Review function
definition and argument passing in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [100]:
rubric_item = 'q14: correct arguments are not passed to `battle` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q14')")[-1])

new_battle = '''
def battle(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'battle', new_battle)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [101]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q15: correct arguments are not passed to `battle` function

In [102]:
rubric_item = 'q15: correct arguments are not passed to `battle` function'
readme_text = """The test verifies if the `battle` function is
called with the correct arguments. Please ensure
that the function is defined correctly and
receives the expected arguments. Review function
definition and argument passing in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [103]:
rubric_item = 'q15: correct arguments are not passed to `battle` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q15')")[-1])

new_battle = '''
def battle(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'battle', new_battle)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [104]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q16: correct arguments are not passed to `battle` function

In [105]:
rubric_item = 'q16: correct arguments are not passed to `battle` function'
readme_text = """The test verifies if the `battle` function is
called with the correct arguments. Please ensure
that the function is defined correctly and
receives the expected arguments. Review function
definition and argument passing in your code."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [106]:
rubric_item = 'q16: correct arguments are not passed to `battle` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q16')")[-1])

new_battle = '''
def battle(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'battle', new_battle)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [107]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### friendship_score: function output is incorrect when the stat difference of the two Pokemon is exactly 20

In [108]:
rubric_item = 'friendship_score: function output is incorrect when the stat difference of the two Pokemon is exactly 20'
readme_text = """Ensure the comparison for stat difference accounts
for a maximum difference of 20 inclusively. Review
conditional checks and the usage of `abs()` in
your implementation."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [109]:
rubric_item = 'friendship_score: function output is incorrect when the stat difference of the two Pokemon is exactly 20'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('friendship_score')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _pkmn1 in project.__pokemon__:
    for _pkmn2 in project.__pokemon__:
        if abs(hidn_get_stat_total(_pkmn1) - hidn_get_stat_total(_pkmn2)) != 20:
            continue
        _pkmn1_types = [project.get_type1(_pkmn1), project.get_type2(_pkmn1)]
        _pkmn2_types = [project.get_type1(_pkmn2), project.get_type2(_pkmn2)]
        if _pkmn1_types[0] in _pkmn2_types:
            continue
        elif _pkmn2_types[0] in _pkmn1_types:
            continue
        elif (_pkmn1_types[1] != "DNE" and _pkmn1_types[1] in _pkmn2_types):
            continue
        elif (_pkmn2_types[1] != "DNE" and _pkmn2_types[1] in _pkmn1_types):
            continue
        var_inputs.append((_pkmn1, _pkmn2))
'''
nb = inject_function_logic_check(nb, 'friendship_score', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### friendship_score: function output is incorrect when the two Pokemon have the same types but not necessarily the same corresponding types

In [110]:
rubric_item = 'friendship_score: function output is incorrect when the two Pokemon have the same types but not necessarily the same corresponding types'
readme_text = """Ensure your code strictly checks for corresponding
`type1` with `type1` and `type2` with `type2`.
Misaligned type comparisons or ignoring the `DNE`
case for `type2` can affect the result. Validate
comparisons for both type order and equality."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [111]:
rubric_item = 'friendship_score: function output is incorrect when the two Pokemon have the same types but not necessarily the same corresponding types'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('friendship_score')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _pkmn1 in project.__pokemon__:
    for _pkmn2 in project.__pokemon__:
        if abs(hidn_get_stat_total(_pkmn1) - hidn_get_stat_total(_pkmn2)) == 20:
            continue
        elif not {project.get_type1(_pkmn1), project.get_type2(_pkmn1)} == {project.get_type1(_pkmn2), project.get_type2(_pkmn2)}:
            continue
        elif [project.get_type1(_pkmn1), project.get_type2(_pkmn1)] == [project.get_type1(_pkmn2), project.get_type2(_pkmn2)]:
            continue
        var_inputs.append((_pkmn1, _pkmn2))
'''
nb = inject_function_logic_check(nb, 'friendship_score', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### friendship_score: function logic is incorrect

In [112]:
rubric_item = 'friendship_score: function logic is incorrect'
readme_text = """Verify that your `friendship_score` function
correctly follows all the rules, especially when
the stat difference is not exactly 20 and when
both types do not match. Ensure proper checks for
'DNE' in `type2` and correct point allocation for
each rule."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [113]:
rubric_item = 'friendship_score: function logic is incorrect'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('friendship_score')")[-1])

var_inputs_code = '''
import project
var_inputs = []
for _pkmn1 in project.__pokemon__:
    for _pkmn2 in project.__pokemon__:
        var_inputs.append((_pkmn1, _pkmn2))
'''
nb = inject_function_logic_check(nb, 'friendship_score', var_inputs_code, 'TEXT_FORMAT')

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))
test_output = results[rubric_item][rubric_item.split(":")[0]]
if test_output != 'All test cases passed!':
    comments[rubric_item] += '\nFAILED TEST CASE: ' + test_output

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q17: correct arguments are not passed to `friendship_score` function

In [114]:
rubric_item = 'q17: correct arguments are not passed to `friendship_score` function'
readme_text = """Check that your `friendship_score` function is
defined correctly and accepts two parameters. Make
sure to pass the correct string arguments when
calling the function."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [115]:
rubric_item = 'q17: correct arguments are not passed to `friendship_score` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q17')")[-1])

false_friendship_score = '''
def friendship_score(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'friendship_score', false_friendship_score)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [116]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q18: correct arguments are not passed to `friendship_score` function

In [117]:
rubric_item = 'q18: correct arguments are not passed to `friendship_score` function'
readme_text = """Check that your `friendship_score` function is
defined correctly and accepts two parameters. Make
sure to pass the correct string arguments when
calling the function."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [118]:
rubric_item = 'q18: correct arguments are not passed to `friendship_score` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q18')")[-1])

false_friendship_score = '''
def friendship_score(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'friendship_score', false_friendship_score)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [119]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q19: correct arguments are not passed to `friendship_score` function

In [120]:
rubric_item = 'q19: correct arguments are not passed to `friendship_score` function'
readme_text = """Check that your `friendship_score` function is
defined correctly and accepts two parameters. Make
sure to pass the correct string arguments when
calling the function."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [121]:
rubric_item = 'q19: correct arguments are not passed to `friendship_score` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q19')")[-1])

false_friendship_score = '''
def friendship_score(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'friendship_score', false_friendship_score)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [122]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### q20: correct arguments are not passed to `friendship_score` function

In [123]:
rubric_item = 'q20: correct arguments are not passed to `friendship_score` function'
readme_text = """Check that your `friendship_score` function is
defined correctly and accepts two parameters. Make
sure to pass the correct string arguments when
calling the function."""

write_readme(readme_text, os.path.join(directories[rubric_item], "README.txt"))

In [124]:
rubric_item = 'q20: correct arguments are not passed to `friendship_score` function'
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('q20')")[-1])

false_friendship_score = '''
def friendship_score(pkmn1, pkmn2):
    return {pkmn1, pkmn2}
'''
nb = replace_with_false_function(nb, 'friendship_score', false_friendship_score)

results[rubric_item] = parse_nb(run_nb(nb, os.path.join(directories[rubric_item], FILE)))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [125]:
gen_public_tests.gen_public_tests(os.path.join(directories[rubric_item], FILE))

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


### general_deductions: Did not save the notebook file prior to running the cell containing "export". We cannot see your output if you do not save before generating the zip file. This deduction will become stricter for future projects.

In [126]:
rubric_item = "general_deductions: Did not save the notebook file prior to running the cell containing \"export\". We cannot see your output if you do not save before generating the zip file. This deduction will become stricter for future projects."
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

results[rubric_item] = {}
results[rubric_item]['general_deductions'] = rubric_item.split(":")[1].strip()
if detect_restart_and_run_all(nb):
    results[rubric_item]['general_deductions'] = "All test cases passed!"

### general_deductions: Functions are defined more than once.

In [127]:
rubric_item = "general_deductions: Functions are defined more than once."
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

results[rubric_item] = {}
results[rubric_item]['general_deductions'] = 'All test cases passed!'

functions = ['damage', 'type_bonus', 'get_num_types', 'effective_damage', 'num_hits', 'battle', 'friendship_score']
redefined_functions = []
for function in functions:
    if count_defns(nb, function) > 1:
        redefined_functions.append(function)
        
if redefined_functions != []:
    results[rubric_item]['general_deductions'] = "following function(s) have multiple definitions:" + repr(list(redefined_functions))    
    comments[rubric_item] = results[rubric_item]['general_deductions']

### general_deductions: Import statements are not all placed at the top of the notebook.

In [128]:
rubric_item = "general_deductions: Import statements are not all placed at the top of the notebook."
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, start=find_all_cell_indices(nb, "markdown", "### Function 1: `damage(attack, defender)`")[0]+1, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

results[rubric_item] = {}
results[rubric_item]['general_deductions'] = 'All test cases passed!'

found_imports = detect_imports(nb)
if found_imports != []:
    results[rubric_item]['general_deductions'] = "found unexpected import(s):" + repr(found_imports)
    comments[rubric_item] = results[rubric_item]['general_deductions']

### general_deductions: Used loops or other material not covered in class yet.

In [129]:
rubric_item = "general_deductions: Used loops or other material not covered in class yet."
nb = clean_nb(read_nb(os.path.join(DIRECTORY, FILE)))
nb = truncate_nb(nb, end=find_all_cell_indices(nb, "code", "grader.check('general_deductions')")[-1])

results[rubric_item] = {}
found_bad_objects = detect_ast_objects(nb, [ast.For, ast.While, ast.List, ast.Dict, ast.Set])
found_imports = set(detect_imports(nb)) - {"otter", "public_tests", "project", "math", "math.ceil", "statistics"}
if found_imports  == set():
    if found_bad_objects == {}:
        results[rubric_item]['general_deductions'] = "All test cases passed!"
    else:
        results[rubric_item]['general_deductions'] = "found unexpected statement(s):\n" + repr(found_bad_objects)
        comments[rubric_item] = results[rubric_item]['general_deductions']
else:
    results[rubric_item]['general_deductions'] = "found unexpected import(s):" + repr(list(found_imports))
    comments[rubric_item] = results[rubric_item]['general_deductions']